# Optmizacion del modelo

In [ ]:
# importamos las librerías a utilizar
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve,roc_curve,confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier, plot_importance

# Configuracion de las rutas para lograr las importaciones

In [ ]:
import sys
import os

# obtiene la ruta absoluta del directorio 'src' desde la ubicación del notebook
src_path = os.path.abspath(os.path.join('..', 'src'))

# agrega la ruta a 'src' al sys.path si no está ya presente
if src_path not in sys.path:
    sys.path.append(src_path)

# Cargamos los datos pre-procesados

In [ ]:
df_hotel = pd.read_csv('./../data/processed/hotel_booking.csv')

In [ ]:
# Mostramos para chequear
df_hotel.head()

In [ ]:
# Mostramos información básica
df_hotel.describe()

In [ ]:
# Mostramos información básica
df_hotel.info()

# Columnas sobre las que trabajar y sus tipo

Guardamos las columnas de cada tipo para poder trabajar mas fácil

In [ ]:
# variable objetivo
target: str = 'is_canceled'

# columnas numéricas
col_numericas: list[str] = ['lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests']

# columnas categoricas
col_categoricas: list[str] = ['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type']

# features
features = col_numericas + col_categoricas

# agrega a las columnas categoricas la variable objetivo
col_categoricas = col_categoricas + [target]

# Separamos los datos en entrenamiento y prueba

In [ ]:
from utils import split_my_data


# Toma las variables y el target
X = df_hotel.drop(columns=target)
y = df_hotel[target]

# divide the dataset into training and test samples
X_train, X_test, y_train, y_test = split_my_data(X, y, test_size=0.2, random_state=42)

# Construccion del modelo XGBoost optimizado

## Inicializamos el modelo

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, f1_score, roc_auc_score


# Inicializamos el modelo
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

## Definimos la estrategia de validacion

In [ ]:
# definir la estrategia de validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Definimos las metricas e evaluacion

In [ ]:
# definir las métricas de evaluación
scoring = {
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True)
}

## Creamos la malla de hiperparametros

In [ ]:
from sklearn.model_selection import GridSearchCV

# definimos la malla de hiperparametros
param_grid = {
    'n_estimators': [100, 300, 500, 700],
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0, 0.1, 0.5, 1],
    'scale_pos_weight': [1, 3, 5, 7]
}

## Configuramos la búsqueda aleatoria con los parametros construidos

In [ ]:
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=50,  # Ajusta el número de iteraciones según tu tiempo
    scoring=scoring,
    cv=cv,
    refit='f1',  # Métrica para seleccionar el mejor modelo
    random_state=42,
    #n_jobs=-1, Utilizar todos los núcleos disponibles
    verbose=2
)

# Hacemos fit del modelo

In [ ]:
random_search.fit(X_train, y_train)

# Evaluación del modelo

In [ ]:
# Para ver los mejores resultados:
print("Mejores hiperparámetros:", random_search.best_params_)
print("Mejor puntaje F1:", random_search.best_score_)

best_model = random_search.best_estimator_

In [ ]:
# Predicciones y evaluación
y_pred = model.predict(X_test)

In [ ]:
y_proba = model.predict_proba(X_test)[:, 1]

## Matriz de confusion

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from draw_utils import draw_confusion_matrix


# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nMatriz de confusion:")
draw_confusion_matrix(conf_matrix)


Interpretación rápida:

- True Neg  **(13713)** → Reservas no canceladas predichas correctamente.
- True Pos  **(7237)** → Cancelaciones correctamente detectadas.
- False Pos **(1194)** → Predijo cancelación, pero la reserva no fue cancelada.
- False Neg **(1734)** → No predijo cancelación, pero sí se canceló.

Con base en estos números, el modelo está haciendo un trabajo razonable, pero hay margen de mejora en los falsos negativos si nuestro objetivo es minimizar el **churn** (posibilidad de que alguien siga con el servicio).

## Metrica ROC AUC

Hacemos el estudio de la metrica ROC AUC para verificar el desempeno del modelo. Se sekecciona esta metrica para el estudio debido a que la data se encuentra desbalanceada.

In [ ]:
from draw_utils import draw_roc_auc


draw_roc_auc(
    y_test=y_test,
    y_prob=y_proba,
    g_title='Curva ROC - Cancelacion de reservacion'
)

## Seleccion del mejor threshold

Para este valor de Auc Roc estudiamos los puntos de corte donde mejor F1-score obtengamos

In [ ]:
# F1 Score por cuantiles
df_eval = pd.DataFrame({'true': y_test, 'proba': y_proba})
# construimos los thresholds a estudiar
thresholds = np.quantile(df_eval['proba'], np.linspace(0.1, 0.9))
scores = []

for t in thresholds:
    # realizamos la prediccion de forma manual con base en
    # las probabilidades que se obtienen del modelo
    pred = (df_eval['proba'] >= t).astype(int)

    #calculamos el f1-score
    f1 = f1_score(df_eval['true'], pred)

    # guardamos el score
    scores.append((t, f1))

print("\n📊 --- F1 Score por punto de corte (cuantiles) ---")
for t, f1 in scores:
    print(f"Threshold: {t:.2f} | F1 Score: {f1:.3f}")

In [ ]:
# seleccionamos el mejor threshold
best_threshold = max(scores, key=lambda x: x[1])[0]

# realizamos las predicciones con este nuevo threshold de clasificacion
y_pred_best_threshold = (y_proba >= best_threshold).astype(int)

print(f"\n✅ Mejor threshold (F1): {best_threshold:.2f}")
print(f"F1 Score: {f1_score(y_test, y_pred_best_threshold):.3f}")
print(f"AUC (el mismo valor visto en la grafica anterior): {roc_auc_score(y_test, y_proba):.3f}")

In [ ]:
# matriz de confusión con el nuevo threshold
conf_matrix_with_opt_f1_score = confusion_matrix(y_test, y_pred_best_threshold)

print("\nMatriz de confusion:")

draw_confusion_matrix(conf_matrix_with_opt_f1_score)

Comparamos ambas matrices de confusion, la original y con el nuevo threshold

In [ ]:
from src.draw_utils import draw_comparison_confusion_matrices

draw_comparison_confusion_matrices(
    confusion_1=conf_matrix,
    confusion_2=conf_matrix_with_opt_f1_score,
    confusion_matrix_1_name='Modelo default',
    confusion_matrix_2_name='Modelo con el threshold que optimiza f1-score'
)

Podemos obsevar que con el nuevo threshold se tiene un mejor desempeno con los falsos postivos, valores extremadamente importantes para el modelo. Por lo tanto es mejor utilizar el mdoelo con el threshold encotnrado que maximisa el f1-score.

### Grafico precision - recall

In [ ]:
from draw_utils import draw_pr_auc


draw_pr_auc(
    y_test=y_test,
    y_prob=y_proba,
    g_title='Curva ROC - Cancelacion de reservacion'
)

Podemos observar que nuestro modelo a tratar de mejorar el Recall disminuye la precision. Por lo tanto, al tratar de mejorar la capacidad del modelo de identificar las personas que cancelaran la capacida de identificar las personas que no cancelan disminuira.

### Interpretacion SHAP

In [ ]:
import shap

# interpretación SHAP
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)

La variable con mas importancia que podemos ver en este gráfico (deposit_type), es la que tiene mas impacto en el modelo, es decir que los depositos que no se realizaron generan mas probabilidad de cancelación

In [ ]:
# Métricas de clasificación
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Podemos observar que el modelo posee buenas métricas con valores de precision, recall y f1-score cercanos por lo tanto parece no existir un alto overfitting.

## Estudio de la importancia de cada variable

In [ ]:
#Interpretación del modelo – Importancia de Variables
import matplotlib.pyplot as plt
from xgboost import plot_importance

# Visualización de importancia de las características
plot_importance(model, max_num_features=10, height=0.5)
plt.title("Importancia de Variables - XGBoost")
plt.tight_layout()
plt.show()

## Interpretación de Importancia de Variables

Las top 3 variables que más influyen en la predicción de cancelaciones son:

1. lead_time (Tiempo entre reserva y llegada) → cuanto mayor, más riesgo de cancelación.
2. adr	(Precio medio por noche) → precios altos pueden ser más susceptibles a cancelación.
3. country	(Origen del huésped) → posiblemente refleja patrones culturales o restricciones.

In [ ]:
# Crear DataFrame con importancias
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
})

# Ordenar por importancia descendente
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)

# Mostrar tabla
feature_importances.reset_index(drop=True, inplace=True)
feature_importances.head(15)


Guardamos la gráfica de feature


 Hallazgos clave de la importancia de variables:

| Variable                   | Interpretación estratégica para churn |
|----------------------------|----------------------------------------|
| `deposit_type`             | La más influyente. Si no hay depósito, el cliente puede cancelar sin penalización. Esto **debería revisarse** como política de negocio. |
| `required_car_parking_spaces` | Clientes que requieren estacionamiento parecen más comprometidos con su estadía. |
| `previous_cancellations`  | Los que han cancelado antes, tienden a hacerlo de nuevo. Perfil de cliente riesgoso. |
| `market_segment`          | El canal de origen de la reserva afecta la tasa de cancelación. Canales online (OTA) suelen tener más cancelaciones. |
| `total_of_special_requests` | Clientes con solicitudes especiales tienden a ser más fieles. |

En cambio, variables como meal, distribution_channel, y assigned_room_type tienen bajo impacto predictivo en este modelo.

## Conclusiones del modelo de cancelación (churn) con XGBoost

**1. Variables más influyentes:**
- `deposit_type`: La política de depósito es el factor más determinante. Las reservas sin depósito tienen alta tasa de cancelación.
- `required_car_parking_spaces`: Los clientes que solicitan estacionamiento parecen estar más comprometidos.
- `previous_cancellations`: El historial del cliente predice comportamiento futuro: quienes ya cancelaron, lo harán de nuevo.

**2. Implicancias de negocio:**
- Reforzar políticas de depósito mínimo o penalización en segmentos con alta cancelación.
- Priorizar promociones hacia segmentos con baja propensión a cancelar (p.ej., quienes hacen solicitudes especiales).
- Evaluar y controlar canales de reserva con alto churn (p.ej., ciertos `market_segment` o `distribution_channel`).

**3. Recomendaciones adicionales:**
- Implementar alertas tempranas para reservas con alto `lead_time` y sin depósito.
- Ofrecer beneficios adicionales a clientes frecuentes que nunca han cancelado (`previous_bookings_not_canceled` alto).
- Reentrenar el modelo regularmente para adaptarse a cambios de comportamiento por estacionalidad o eventos externos.

---

✅ El modelo XGBoost ofrece una buena capacidad predictiva y guía acciones concretas para reducir la tasa de cancelaciones.
